# **EDF PROJECT**

This is our report for the EDF Project. The team is composed of four members:
*   Hugo Tran
*   Akira Kuroda
*   Tianhe Li
*   Etienne Guegan





Before getting into the core of the project, we need to import the following libraries

### **Description of approach and the models used**

We observed that the data given of cell C and D contains the total time, current, voltage and step. We decided to plot the current, voltage and step in function of the time elapsed.

We made a function ```plot_test(cell, test)``` that takes two arguments (both in ```String```). Here is the plot of cell D with test 9:

In [2]:
pt.plot_test("D", "09")


NameError: name 'pt' is not defined

We get the following result:

<div align = "center">
    <img src = "img/data_cell_d_test_09.png" alt = "Data plot of Test 9 of Cell D in function of the total time" width = "900"/>
</div>

Our goal is to build an electrical model of the cell based on ECM that is a function of the SoC and SoH. So it translates to finding the output voltage $U_L$ of the battery in function of the SoC.
During the first week of the project, we used the Rint Model of the battery (0th order model). 
<div align = "center">
<img src = "img/rint_model.png" alt = "Rint Model circit" width = "500"/>
</div>

Its equation of $U_L$ is:

\begin{equation}
\tag{1}
U_L = U_{OC} + U_0
\implies U_L = U_{OC} + I_L \cdot R_{0}
\end{equation}


It is a simple approach get the output voltage, since we just need to calculate the $U_{OC}$ and the internal resistance $R_0$.

#### Finding $U_{OC}$

From the resource given, $U_{OC}$, or Open Circuit Voltage, is found when the battery has been at rest for a long enough time (1 or 2 hours). 
This translates graphically to finding the voltage of the battery when its current is equal to ```0 Amps```. For instance for cell D test 9, the circled portions of the graphs correspond to its OCV.

<div align = "center">
<img src = "img/finding_ocv.png" alt = "Finding OCV" width = "700"/>
</div>

Below is the code of the dataframe of cell D on test 9, where we keep the values such that its current is equal to ```0 Amps```.


In [4]:
print(dt.find_OCV("D", "09")) # Dataframe with OCV extracted

print(dt.soc_ocv("D", "09")) # Dataframe with OCV extracted, and SoC calculated


NameError: name 'dt' is not defined

However, since we can only measure the voltage after a pulse, we only get a sample of the OCV. So we can only establish a discrete relationship between the OCV and the SoC. 

To get its continuous version, we apply curve fitting thanks to the ```np.polyfit``` function.
We therefore have the following code:

In [ ]:
soc = soc_ocv(cell, test)["SoC"] # Extract the OCV from the dataframe
ocv = soc_ocv(cell, test)["OCV"] # Extract the SoC from the dataframe

# Fit a polynomial of degree 5
coefficients = np.polyfit(soc, ocv, 5) # Curve fitting the SoC in function of the OCV by a 5th degree polynomial 
polynomial = np.poly1d(coefficients) # Store the coefficients in the polynomial

# Thus, given an OCV, we can find the SoC by executing: polynomial(OCV)


So we finally get $U_{OC}$ in function of the SoC.

#### Finding $R_0$

From the Rint Model, we observe that: $U_L = U_{OC} + U_0 \implies U_L - U_{OC} = U_0$

$U_L$ is given, while $U_{OC}$ was calculated. We know that $U_0 = I_L \cdot R_0 \implies R_0 = \frac{U_0}{I_L} \implies R_0 = \frac{U_L - U_{OC}}{I_L}$. 

The following code adds $R_0$ to the dataframe of cell D on test 9:

In [ ]:
add_R0("D", "09")


### **SoH plot of both cells**

The State of Health (SoH) of batterries correspond to the ratio of the remaining capacity at a given time to the initial capacity. In other words, it shows how much capacity the battery has lost due to ageing. The SoH is expressed in percentage by the following equation:

\begin{equation}
\tag{2}
SoH [\%] = \frac{Q_{remaining}}{Q_{initial}}
\end{equation}

We decided to plot the SoH of cells C and D in function of their test value. Cell C has 23 tests, while cell D has 13. Test 00 is be the start of the experiment, meaning that the batteries are new in this test.


The following function ```plot_soh(cell)``` from the ```plot``` file takes ```"C"``` or ```"D"``` as argument. We plot the SoH of cell C:

In [9]:
pt.plot_soh("C")


NameError: name 'pt' is not defined

We get the following images:

<div align = "center">
    <img src = "img/soh_c.png" alt = "Image of SoH of Cell C in function of the tests" width = "575"/>
    <img src = "img/soh_d.png" alt = "Image of SoH of Cell D in function of the tests" width = "575"/>
</div>
